In [8]:
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd
import numpy as np
from scipy.stats import poisson
import json
import sqlite3
import calendar
import datetime
from dateutil.relativedelta import relativedelta
import datetime
import os

SPREADSHEET_IDS = {
    "EXA FIRST": "10-B_vV1pvUzXmvGAiHhODGJgCloOsAmqSO9HvXpk_T8",
    "コンサートホールエフ成増": "1EDY2RfjDQNsapVrl2X-UrqPKoXrkQmYJnk3uPqccBxY",
    "第一プラザ坂戸1000": "170MVr-BB3LG-g5ItkDT-8TE6R68RW9zJhRfpvQiy-PE",
    "第一プラザみずほ台店": "1_1722pigi_Z1D6eH0tsPfMneGoS9O09fyqD6F-h1mQA",
    "パールショップともえ川越店": "1i70joJ27Hs7inS-D89z9YMSJO1aRvaBeeWn0n9xpktY",
    "パラッツォ川越店": "179nJF0NvLng7xPKsd_NX2pJBXsDNsO8SJhOvUAvFk2I",
    "第一プラザ狭山店": "1IVb2Woq3n_PDZP87LdW9NpFP3Z6LeyQtONCkx_fWIq4",
    "ニュークラウン川越2号店": "1jocaH94-5GKWUqEqcrybgFcpb1Tn0BhvLMqaADppNWg",
    "TOHO川越店": "1VOr7BNMB-xnHihZrk6MFmetlZ0nCFP86YcrJu9w0dac",
}
# スプレッドシート認証設定
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
jsonf = r"C:\python\dataOnline\anaslo_02\json\spreeadsheet-347321-ff675ab5ccbd.json"
creds = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
client = gspread.authorize(creds)

In [9]:
def create_df_from_database(hall_name, start_date, end_date, model_name=None):
    # Table name 取得
    DB_PATH = r"C:\python\dataOnline\anaslo_02\db\anaslo_02.db"
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
    tables = cursor.fetchall()
    # print(tables)

    cursor.execute(
        "SELECT hall_id, name FROM halls WHERE name LIKE ?", ("%" + hall_name + "%",)
    )
    results = cursor.fetchall()

    # 結果表示
    if results:
        hall_id, hall_name = results[0]
        print(f"🔍 '{hall_name}' を含むホール名が見つかりました。")
    else:
        print(f"❌ '{hall_name}' を含むホール名は見つかりませんでした。")

    query = """
        -- 出玉データにホール名と機種名を結合して取得
        SELECT
            r.*, 
            h.name AS hall_name,
            m.name AS model_name
        FROM results r
        JOIN halls h ON r.hall_id = h.hall_id
        JOIN models m ON r.model_id = m.model_id
        WHERE h.name = ?
        AND r.date BETWEEN ? AND ?
        """

    params = [hall_name, start_date, end_date]
    if model_name:
        query += " AND m.name LIKE ?"
        params.append(f"%{model_name}%")  # 部分一致にする

    query += " ORDER BY r.date DESC, r.unit_no ASC"

    df = pd.read_sql_query(query, conn, params=params)
    print(f"データサイズ: {df.shape[0]} x {df.shape[1]}")
    print(f"📅 検索期間: {start_date} ～ {end_date}", f"📅 抽出期間: {df.date.min()} ～ {df.date.max()}")
    print(f'含まれる日数 : {df["date"].nunique()}')

    return df

In [58]:
# ブドウシミュレーター
def grape_calc_myfive(game, bb, rb, medals, cherry=True):
    bb_medals = 239.25
    rb_medals = 95.25
    replay_rate = 0.411
    if cherry:
        cherry_rate_high = 0.04228
    else:
        cherry_rate_high = 0.05847
    denominator_inner = (
        -medals
        - (
            game * 3
            - (
                bb * bb_medals
                + rb * rb_medals
                + game * replay_rate
                + game * cherry_rate_high
            )
        )
    ) / 8
    grape_rate = (game / denominator_inner) - ((game / denominator_inner) * 2)
    return grape_rate


GRAPE_CONSTANTS = {
    "マイジャグラーV": {
        "bb": 239.25,
        "rb": 95.25,
        "replay": 0.411,
        "cherryOff": 0.05847,
        "cherryOn": 0.04228,
    },
    "アイムジャグラーEX-TP": {
        "bb": 251.25,
        "rb": 95.25,
        "replay": 0.411,
        "cherryOff": 0.06068,
        "cherryOn": 0.040475,
    },
    "ゴーゴージャグラー3": {
        "bb": 239.00,
        "rb": 95.00,
        "replay": 0.411,
        "cherryOff": 0.0661,
        "cherryOn": 0.0372,
    },
    "ファンキージャグラー2": {
        "bb": 239.25,
        "rb": 95.25,
        "replay": 0.411,
        "cherryOff": 0.0603,
        "cherryOn": 0.04324,
    },
}


def calc_grape_rate(row, constants, cherry=True):
    model = row["model_name"]
    if model not in constants:
        return None
    try:
        game = row["game"]
        bb = row["BB"]
        rb = row["RB"]
        medals = row["medals"]
        # 定数取得
        c = constants[model]
        cherry_rate = c["cherryOn"] if cherry else c["cherryOff"]
        # 分母計算式
        denominator = (
            -medals
            - (
                game * 3
                - (
                    bb * c["bb"]
                    + rb * c["rb"]
                    + game * c["replay"]
                    + game * cherry_rate
                )
            )
        ) / 8
        if denominator == 0:
            return None  # ゼロ除算防止
        grape = (game / denominator) - ((game / denominator) * 2)
        return round(grape, 2)

    except Exception as e:
        print(f"⚠️ Grape計算失敗: {model} → {e}")
        return None


def assign_area(unit_no, json_file_path):
    with open(json_file_path, "r", encoding="utf-8") as f:
        area_map = json.load(f)
    for rule in area_map:
        if rule["start"] <= unit_no <= rule["end"]:
            return rule["area"]
    return "other"


jagglar_rate_path = r"C:\python\dataOnline\anaslo_02\json\jagglar_rate.json"
with open(jagglar_rate_path, "r", encoding="utf-8") as f:
    MODEL_DATA_DICT = json.load(f)


# 設定推定関数
def estimate_setting_probs(row, limit_games=5000):
    # JSONファイルの確率表を読み込む
    model = row["model_name"]
    games = row["game"]
    if games < limit_games or pd.isna(row["RB_rate"]) or pd.isna(row["Total_rate"]):
        return {f"設定{i}": 0.0 for i in range(1, 7)}
    use_grape = not pd.isna(row["Grape_rate"]) and row["Grape_rate"] != 0
    rb_actual = row["RB"]
    total_actual = row["BB"] + row["RB"]
    if use_grape:
        grape_actual = round(games / row["Grape_rate"])

    model_data = MODEL_DATA_DICT.get(model)
    if not model_data:
        return {f"設定{i}": 0.0 for i in range(1, 7)}

    likelihoods = {}
    for setting, values in model_data.items():
        try:
            rb_rate = float(values["RB_RATE"].split("/")[1])
            total_rate = float(values["TOTAL_RATE"].split("/")[1])
            grape_rate = float(values["GRAPE_RATE"].split("/")[1])
            expected_rb = games / rb_rate
            expected_total = games / total_rate
            expected_grape = games / grape_rate
            rb_likelihood = poisson.pmf(rb_actual, expected_rb)
            total_likelihood = poisson.pmf(total_actual, expected_total)
            # Grapeを使うかどうか
            if use_grape:
                grape_rate = float(values["GRAPE_RATE"].split("/")[1])
                expected_grape = games / grape_rate
                grape_likelihood = poisson.pmf(grape_actual, expected_grape)
                likelihood = rb_likelihood * total_likelihood * grape_likelihood
            else:
                likelihood = rb_likelihood * total_likelihood

            likelihoods[int(setting)] = likelihood
        except:
            continue
    total_sum = sum(likelihoods.values())
    if total_sum == 0:
        return {f"設定{i}": 0.0 for i in range(1, 7)}
    probs = {f"設定{s}": round(l / total_sum, 2) for s, l in likelihoods.items()}
    return probs

def df_preprocessing(df, hall_name):
    json_path = f"C:/python/dataOnline/anaslo_02/json/{hall_name}_area_map.json"
    if not os.path.exists(json_path):
        json_path = f"C:/python/dataOnline/anaslo_02/json/other_area_map.json"
    print(f"データ前処理を行います")
    df_pre = df.copy()
    df_pre["date"] = pd.to_datetime(df_pre["date"])
    df_pre.drop(columns=["result_id", "hall_id", "model_id"], inplace=True)
    df_pre_columns = ["hall_name", "date", "model_name", "unit_no", "game", "BB", "RB", "medals"]
    df_pre = df_pre[df_pre_columns]
    df_pre["BB_rate"] = (df_pre["game"] / df_pre["BB"]).round(1)
    df_pre["RB_rate"] = (df_pre["game"] / df_pre["RB"]).round(1)
    df_pre["Grape_rate"] = df_pre.apply(lambda row: calc_grape_rate(row, GRAPE_CONSTANTS), axis=1)
    df_pre["Total_rate"] = (df_pre["game"] / (df_pre["BB"] + df_pre["RB"])).round(1)
    
    df_pre["month"] = df_pre["date"].dt.strftime("%Y-%m")
    df_pre["day"] = df_pre["date"].dt.day
    df_pre["weekday"] = df_pre["date"].dt.weekday
    df_pre["year"] = df_pre["date"].dt.year
    df_pre["unit_last"] = df_pre["unit_no"].astype(str).str[-1]
    df_pre["last_digit"] = df_pre["date"].dt.day.astype(str).str[-1].astype(int)
    df_pre["area"] = df_pre["unit_no"].apply(lambda x: assign_area(x, json_path))

    # 設定推定
    df_probs = df_pre.apply(estimate_setting_probs, axis=1, result_type="expand")
    df_pre["5more"] = df_probs[["設定5", "設定6"]].sum(axis=1)
    
    df_pre.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_pre = df_pre.fillna(0)

    model_list = list(df["model_name"].unique())

    return df_pre, model_list


def add_spreadsheet(spreadsheet, sheet_name, df, sheet_clear=False):
    today = datetime.datetime.today()
    rows, cols = df.shape
    try:
        sheet = spreadsheet.worksheet(sheet_name)
        print(f"✅ シート「{sheet_name}」が既に存在します。")
    except gspread.exceptions.WorksheetNotFound:
        sheet = spreadsheet.add_worksheet(
            title=sheet_name, rows=df.shape[0]+3, cols=df.shape[1]+3
        )
        print(f"🆕 シート「{sheet_name}」を新規作成しました。")
    if sheet_clear:
        sheet.clear()
    last_row = len(sheet.get_all_values())
    set_with_dataframe(sheet, df, row=last_row, include_index=True)
    sheet.update_cell(1, 1, today.strftime("%Y-%m-%d UPDATED"))
    print(f"✅ シート '{sheet_name}' に DataFrame を書き込みました！")

## ピボットテーブル作成


In [11]:
def create_pivot_table(
    df,
    index,
    columns,
    pivots=["game", "medals", "BB", "RB"],
    reverse=False,
    margins=True,
    day_target=None,
):
    df_filtered = df.copy()
    if day_target is not None:
        df_filtered = df_filtered[df_filtered["day"] == day_target]

    pivot_results = {}
    for col in pivots:
        table = df_filtered.pivot_table(
            index=index,
            columns=columns,
            values=col,
            aggfunc="sum",
            margins=margins,
            margins_name="total",
        )
        pivot_results[col] = table
        if reverse:
            pivot_results[col] = table.iloc[:, ::-1]

    game = pivot_results["game"]
    medals = pivot_results["medals"]
    rb = pivot_results["RB"]
    bb = pivot_results["BB"]
    rb_rate = (game / rb).round(1)
    total_rate = (game / (bb + rb)).round(1)
    medal_rate = ((medals + game * 3) / (game * 3)).round(3)

    labeled_tables = [
        ("GAME", game),
        ("MEDALS", medals),
        ("RB_RATE", rb_rate),
        ("TOTAL_RATE", total_rate),
        ("MEDAL_RATE", medal_rate),
        ("BB", bb),
        ("RB", rb),
    ]

    # ラベルを MultiIndex に付ける
    for label, df_table in labeled_tables:
        df_table.columns = pd.MultiIndex.from_product([[label], df_table.columns])

    # 列を交互に整列して統合
    interleaved_cols = [
        col
        for pair in zip(
            game.columns,
            medals.columns,
            bb.columns,
            rb.columns,
            medal_rate.columns,
            rb_rate.columns,
            total_rate.columns,
        )
        for col in pair
    ]
    merged = pd.concat([game, medals, medal_rate, bb, rb, rb_rate, total_rate], axis=1)[
        interleaved_cols
    ]
    merged.replace([np.inf, -np.inf, np.nan], None, inplace=True)
    details = {
        "game": game,
        "medals": medals,
        "medal_rate": medal_rate,
        "bb": bb,
        "rb": rb,
        "rb_rate": rb_rate,
        "total_rate": total_rate,
    }

    return merged, details



## 出力データ

- 期間指定
  - 機種別 x 日付
  - 島別 x 日付
  - 台番号 x 日付
  - 島別 x 月
  - 過去 n 日の差枚比較

In [7]:
hall_name = "EXA FIRST"
# hall_name = "コンサートホールエフ成増"
# hall_name = "第一プラザみずほ台店"
# hall_name = "オータ志木駅前店"
# hall_name = "第一プラザ坂戸1000"
# hall_name = "ニュークラウン川越2号店"
# hall_name = "TOHO川越店"
# hall_name = "グランドオータ新座駅前店"
# hall_name = "ニューダイエイiii"

model_name = "ジャグラー"

today = datetime.date.today()
start_date = today - relativedelta(months=6, days=today.day - 1)
df_db = create_df_from_database(hall_name, start_date, today, model_name=model_name)
df, model_list = df_preprocessing(df_db, hall_name)
df.to_csv("df_preprocessing.csv", index=False, encoding="utf-8-sig")
spreadsheet = client.open_by_key(SPREADSHEET_IDS[hall_name])
print(f"スプレッドシート: {spreadsheet.title} を開きました。")
df.tail()

🔍 'EXA FIRST' を含むホール名が見つかりました。
データサイズ: 31616 x 11
📅 検索期間: 2024-11-01 ～ 2025-05-30 📅 抽出期間: 2024-11-01 ～ 2025-05-29
含まれる日数 : 208
データ前処理を行います
スプレッドシート: EXA FIRST を開きました。


,hall_name,date,model_name,unit_no,game,BB,RB,medals,BB_rate,RB_rate,Grape_rate,Total_rate,month,day,weekday,year,unit_last,area,5more
31611,EXA FIRST,2024-11-01,アイムジャグラーEX-TP,1151,7065,24,22,456,294.4,321.1,5.47,153.6,2024-11,1,4,2024,1,u: アイムジャグラーEX-TP,0.80
31612,EXA FIRST,2024-11-01,アイムジャグラーEX-TP,1152,6208,26,20,1041,238.8,310.4,5.90,135.0,2024-11,1,4,2024,2,u: アイムジャグラーEX-TP,0.37
31613,EXA FIRST,2024-11-01,アイムジャグラーEX-TP,1153,5987,16,22,-679,374.2,272.1,5.66,157.6,2024-11,1,4,2024,3,u: アイムジャグラーEX-TP,0.62
31614,EXA FIRST,2024-11-01,アイムジャグラーEX-TP,1154,7888,28,35,1206,281.7,225.4,5.77,125.2,2024-11,1,4,2024,4,u: アイムジャグラーEX-TP,0.96
31615,EXA FIRST,2024-11-01,アイムジャグラーEX-TP,1155,7277,25,32,1065,291.1,227.4,5.66,127.7,2024-11,1,4,2024,5,u: アイムジャグラーEX-TP,0.97


## MODEL_RATE_DAY


In [12]:
sheet_name = "MODEL_RATE"
csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
index = ["model_name"]
columns = ["day"]
merged, details = create_pivot_table(df, index, columns, reverse=False, margins=True)
model_rate = details["medal_rate"].copy()
model_rate.to_csv(csv_path)
# add_spreadsheet(spreadsheet, sheet_name, model_rate, sheet_clear=True)
model_rate.head()

MEDAL_RATE                                                   \
day                    1      2      3      4      5      6      7      8   
model_name                                                                  
アイムジャグラーEX-TP      1.007  0.988  1.002  0.980  1.002  1.001  0.985  1.002   
ウルトラミラクルジャグラー      1.010  1.010  1.007  0.998  1.008  0.994  1.019  0.983   
ゴーゴージャグラー3         1.007  1.011  0.980  0.995  1.009  1.008  0.999  0.994   
ジャグラーガールズ          1.007  1.000  0.976  0.967  0.996  1.001  0.993  0.985   
ハッピージャグラーVIII      0.993  0.959  0.933  1.020  0.997  1.003  1.029  1.056   

                             ...                                            \
day                9     10  ...     23     24     25     26     27     28   
model_name                   ...                                             
アイムジャグラーEX-TP  0.981  0.999  ...  1.005  0.983  0.996  1.023  0.980  0.998   
ウルトラミラクルジャグラー  0.986  0.987  ...  0.988  0.986  1.004  1.030  0.998  0.981   
ゴーゴージャグラー3     0.998  1.003  ...  0.996  0.995  0.996  1.015  0.994  0.975   
ジャグラーガールズ      0.987  0.998  ...  0.992  1.002  1.005  1.003  0.987  0.985   
ハッピージャグラーVIII  0.985  1.002  ...  0.963  0.958  0.993  1.026  0.949  1.002   

                                           
day               29     30     31  total  
model_name                                 
アイムジャグラーEX-TP  0.993  1.000  0.989  0.998  
ウルトラミラクルジャグラー  0.990  0.970  0.969  0.997  
ゴーゴージャグラー3     0.987  0.999  0.989  0.998  
ジャグラーガールズ      0.990  0.986  0.993  0.995  
ハッピージャグラーVIII  0.964  0.901    NaN  0.990  

[5 rows x 32 columns]

## ISLAND_RATE


In [13]:
sheet_name = f"ISLAND_RATE"
csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
index = ["area"]
columns = ["day"]
merged, details = create_pivot_table(df, index, columns, reverse=False, margins=True)
island_rate = details["medal_rate"].copy()
island_rate.to_csv(csv_path)
# add_spreadsheet(spreadsheet, sheet_name, island_rate, sheet_clear=True)
island_rate.head()

MEDAL_RATE                                                   \
day                    1      2      3      4      5      6      7      8   
area                                                                        
a: ゴーゴージャグラー3      1.039  0.991  1.003  0.998  1.005  1.010  1.002  0.980   
b: ゴーゴージャグラー3      1.019  0.994  0.988  1.010  1.010  1.022  1.001  0.995   
c: マイジャグラーV        1.054  0.981  0.982  1.032  1.001  0.991  1.004  0.988   
d: マイジャグラーV        0.999  0.980  1.005  0.996  1.022  1.000  0.995  0.998   
e: マイジャグラーV        1.032  1.011  0.999  0.992  1.027  1.002  1.027  1.009   

                             ...                                            \
day                9     10  ...     23     24     25     26     27     28   
area                         ...                                             
a: ゴーゴージャグラー3  1.000  1.015  ...  1.000  0.994  0.995  1.010  1.025  0.976   
b: ゴーゴージャグラー3  0.979  0.986  ...  0.998  1.016  1.012  1.013  1.009  0.974   
c: マイジャグラーV    1.006  1.000  ...  0.993  0.993  1.006  0.994  0.999  0.983   
d: マイジャグラーV    0.973  0.990  ...  0.990  0.983  1.022  1.025  0.995  1.000   
e: マイジャグラーV    0.977  0.996  ...  1.014  1.008  1.004  0.993  1.006  0.990   

                                           
day               29     30     31  total  
area                                       
a: ゴーゴージャグラー3  0.989  0.982  1.004  1.004  
b: ゴーゴージャグラー3  0.997  0.979  0.981  1.002  
c: マイジャグラーV    1.003  0.989  1.004  1.001  
d: マイジャグラーV    0.984  0.999  1.014  1.003  
e: マイジャグラーV    0.990  0.999  1.031  1.006  

[5 rows x 32 columns]

## UNIT_RATE


In [14]:
sheet_name = f"UNIT_RATE"
csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
index = ["model_name", "unit_no"]
columns = ["day"]
merged, details = create_pivot_table(df, index, columns, reverse=False, margins=True)
target_rate = 1.05
unit_rate = details["medal_rate"].copy()
unit_rate[("MEDAL_RATE", f"{target_rate}+")] = (
    unit_rate.iloc[:, :-1] >= target_rate
).sum(axis=1)
countif = (unit_rate.iloc[:-1, :] >= target_rate).sum(axis=0)
unit_rate = pd.concat(
    [unit_rate, pd.DataFrame([countif], index=[(f"count_{target_rate}+", "")])], axis=0
)
unit_rate.replace([np.inf, -np.inf], np.nan, inplace=True)
unit_rate = unit_rate[~unit_rate.iloc[:, 1].isna()]
unit_rate.to_csv(csv_path)
# add_spreadsheet(spreadsheet, sheet_name, unit_rate, sheet_clear=True)
unit_rate.head()

MEDAL_RATE                                            \
day                         1      2      3      4      5      6      7   
アイムジャグラーEX-TP 1100      0.877  0.958  0.995  0.919  1.098  1.105  0.783   
              1101      1.054  0.848  1.103  0.929  0.965  0.900  0.922   
              1102      1.018  1.064  1.030  0.910  0.950  1.066  0.861   
              1103      1.000  1.184  0.986  0.719  1.024  1.059  0.876   
              1104      1.042  0.977  0.878  1.061  0.904  1.124  0.740   

                                         ...                              \
day                     8      9     10  ...     24     25     26     27   
アイムジャグラーEX-TP 1100  0.809  0.705  1.019  ...  0.977  0.997  0.957  1.016   
              1101  0.948  0.862  1.081  ...  0.826  0.880  1.019  1.143   
              1102  0.862  0.902  1.109  ...  0.917  1.010  1.086  0.960   
              1103  0.974  0.710  0.821  ...  0.951  0.968  1.002  0.953   
              1104  0.903  0.722  0.931  ...  1.012  0.910  0.991  0.988   

                                                          
day                    28     29     30  31  total 1.05+  
アイムジャグラーEX-TP 1100  1.069  1.017  1.070 NaN  0.993     5  
              1101  1.075  0.989  0.899 NaN  1.019     9  
              1102  0.912  0.987  0.938 NaN  1.010     7  
              1103  1.116  0.921  1.044 NaN  1.000     6  
              1104  1.068  1.001  0.907 NaN  0.995     5  

[5 rows x 33 columns]

## LAST_DIGIT_RATE

In [15]:
sheet_name = f"LAST_DIGIT_RATE"
csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
index = ["model_name"]
columns = ["last_digit"]
merged, details = create_pivot_table(df, index, columns, reverse=False, margins=True)
last_digit_rate = details["medal_rate"]
last_digit_rate.replace([np.inf, -np.inf], np.nan, inplace=True)
last_digit_rate = last_digit_rate[~last_digit_rate.iloc[:, 1].isna()]
last_digit_rate.to_csv(csv_path)
add_spreadsheet(spreadsheet, sheet_name, last_digit_rate, sheet_clear=True)


KeyError: 'last_digit'

# DAYn

- SHEET_NAME : DAY1，DAY2，… DAY31


In [16]:
index = ["area", "unit_no"]
columns = ["date"]
day_list = [21]
# for day_target in range(today.day - 1, today.day + 1):
for day_target in day_list:
    sheet_name = f"DAY{day_target}"
    csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
    merged, details = create_pivot_table(df, index, columns, reverse=True, margins=True, day_target=day_target)
    merged.to_csv(csv_path)
    # add_spreadsheet(spreadsheet, f"DAY{day_target}", merged, sheet_clear=True,)
merged.head()

GAME MEDALS    BB    RB MEDAL_RATE RB_RATE TOTAL_RATE  \
date                   total  total total total      total   total      total   
area          unit_no                                                           
a: ゴーゴージャグラー3 1001     28047  -4119    94    73      0.951   384.2      167.9   
              1002     32615   2262   134    88      1.023   370.6      146.9   
              1003     43885   3069   184   132      1.023   332.5      138.9   
              1004     32198    -16   117   100      1.000   322.0      148.4   
              1005     32151   4665   138    96      1.048   334.9      137.4   

                                     GAME              MEDALS  \
date                  2025-05-21 00:00:00 2025-05-21 00:00:00   
area          unit_no                                           
a: ゴーゴージャグラー3 1001                   5013                -768   
              1002                   2270                -209   
              1003                   8640                 -88   
              1004                   1857                 232   
              1005                   4248                -332   

                                       BB  ...          MEDAL_RATE  \
date                  2025-05-21 00:00:00  ... 2024-12-21 00:00:00   
area          unit_no                      ...                       
a: ゴーゴージャグラー3 1001                     16  ...               1.005   
              1002                      8  ...               0.957   
              1003                     30  ...               1.008   
              1004                      9  ...               0.843   
              1005                     15  ...               1.081   

                                  RB_RATE          TOTAL_RATE  \
date                  2024-12-21 00:00:00 2024-12-21 00:00:00   
area          unit_no                                           
a: ゴーゴージャグラー3 1001                  446.5               162.4   
              1002                  475.6               184.9   
              1003                  386.1               149.7   
              1004                  446.4               248.0   
              1005                  292.7               127.3   

                                     GAME              MEDALS  \
date                  2024-11-21 00:00:00 2024-11-21 00:00:00   
area          unit_no                                           
a: ゴーゴージャグラー3 1001                   6954                -362   
              1002                   3472                -397   
              1003                   9028                2197   
              1004                   7887                1335   
              1005                   1418                -815   

                                       BB                  RB  \
date                  2024-11-21 00:00:00 2024-11-21 00:00:00   
area          unit_no                                           
a: ゴーゴージャグラー3 1001                     26                  21   
              1002                     12                   8   
              1003                     42                  32   
              1004                     31                  28   
              1005                      4                   1   

                               MEDAL_RATE             RB_RATE  \
date                  2024-11-21 00:00:00 2024-11-21 00:00:00   
area          unit_no                                           
a: ゴーゴージャグラー3 1001                  0.983               331.1   
              1002                  0.962               434.0   
              1003                  1.081               282.1   
              1004                  1.056               281.7   
              1005                  0.808              1418.0   

                               TOTAL_RATE  
date                  2024-11-21 00:00:00  
area          unit_no                      
a: ゴーゴージャグラー3 1001                  148.0  
              1002               

## MONTH_RATE

In [17]:
start_date = today - relativedelta(months=100, days=today.day-1)
df_db = create_df_from_database("EXA FIRST", start_date, today, model_name=model_name)
df, model_list = df_preprocessing(df_db, hall_name)
sheet_name = "MONTH_RATE"
csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
index = ["model_name"]
columns = ["month"]
merged, details = create_pivot_table(df, index, columns, reverse=True, margins=True)
month_rate = details["medal_rate"].copy()
# add_spreadsheet(spreadsheet, sheet_name, month_rate, sheet_clear=True)
month_rate.to_csv(csv_path)
month_rate

🔍 'EXA FIRST' を含むホール名が見つかりました。
データサイズ: 64144 x 11
📅 検索期間: 2017-01-01 ～ 2025-05-30 📅 抽出期間: 2024-04-01 ～ 2025-05-29
含まれる日数 : 422
データ前処理を行います


MEDAL_RATE                                                  \
month              total 2025-05 2025-04 2025-03 2025-02 2025-01 2024-12   
model_name                                                                 
アイムジャグラーEX-TP      1.001   0.997   0.998   1.001   0.993   0.997   1.000   
ウルトラミラクルジャグラー      0.997   0.987   0.993   0.997   1.002   1.000   1.001   
ゴーゴージャグラー3         1.002   0.999   0.993   0.992   1.001   1.000   0.996   
ジャグラーガールズ          1.003   0.995   0.993   1.001   0.990   0.996   0.994   
ハッピージャグラーVIII      0.995   0.991   0.995     NaN     NaN     NaN     NaN   
ファンキージャグラー2        0.996   0.993   0.992   0.993   0.996   0.990   0.995   
マイジャグラーV           1.010   1.003   1.000   1.002   1.003   1.006   1.005   
ミスタージャグラー          1.001   0.996   1.002   0.991   0.996   0.990   1.006   
total              1.006   0.999   0.998   0.999   1.001   1.002   1.002   

                                                                               
month         2024-11 2024-10 2024-09 2024-08 2024-07 2024-06 2024-05 2024-04  
model_name                                                                     
アイムジャグラーEX-TP   1.001   0.999   1.001   1.000   1.007   1.005   1.004   1.008  
ウルトラミラクルジャグラー     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN  
ゴーゴージャグラー3      1.005   0.995   0.998   1.003   1.003   1.006   1.010   1.012  
ジャグラーガールズ       0.995   0.999   1.003   0.997   1.005   1.005   1.010   1.016  
ハッピージャグラーVIII   0.957   1.004     NaN     NaN     NaN     NaN     NaN     NaN  
ファンキージャグラー2     0.996   1.000   0.989   1.001   0.986   0.990   1.000   1.007  
マイジャグラーV        1.002   1.008   1.009   1.012   1.011   1.017   1.021   1.025  
ミスタージャグラー       0.997   1.002   1.008   1.007   1.009     NaN     NaN     NaN  
total           1.001   1.004   1.005   1.007   1.008   1.011   1.015   1.018

## HISTORY

In [18]:
start_date = datetime.date(today.year, today.month-1, 1) - relativedelta(days=6)
df_db = create_df_from_database(hall_name, start_date, today, model_name="ジャグラー")
df, model_list = df_preprocessing(df_db, hall_name)


pivot_targets = ["medals", "game", "RB_rate", "Total_rate", "Grape_rate"]
index_targets = ["area", "model_name", "unit_no"]
columns_targets = ["date"]

merged_by_model = pd.DataFrame()
for model in df["model_name"].unique():
    
    # モデルごとにデータをフィルタリング
    df_model = df[df["model_name"] == model]
    pivot_results = {}
    for col in pivot_targets:
        table = df_model.pivot_table(
            index=index_targets,
            columns=columns_targets,
            values=col,
            aggfunc="sum",
        )
        pivot_results[col] = table
    
    medals = pivot_results["medals"]
    game = pivot_results["game"]
    rb_rate = pivot_results["RB_rate"]
    total_rate = pivot_results["Total_rate"]
    grape_rate = pivot_results["Grape_rate"]
    medal_rate = ((medals + game * 3) / (game * 3)).round(3)
    
    # ランキング列作成
    rolling7 = medals.T.rolling(7, min_periods=7).sum().T
    medal_rank7 = rolling7.rank(method="min", ascending=True)
    medal_rank7 = medal_rank7.fillna(0).replace([np.inf, -np.inf], 0).astype(int)
    rolling5 = medals.T.rolling(5, min_periods=5).sum().T
    medal_rank5 = rolling5.rank(method="min", ascending=True)
    medal_rank5 = medal_rank5.fillna(0).replace([np.inf, -np.inf], 0).astype(int)
    rolling3 = medals.T.rolling(3, min_periods=3).sum().T
    medal_rank3 = rolling3.rank(method="min", ascending=True)
    medal_rank3 = medal_rank3.fillna(0).replace([np.inf, -np.inf], 0).astype(int)
    medal_rank1 = medals.rank(method="min", ascending=True)
    medal_rank1 = medal_rank1.fillna(0).replace([np.inf, -np.inf], 0).astype(int)

    # MultiIndex化（ラベル付け）
    labeled_tables = [
        ("GRAPE_RATE", grape_rate),
        ("TOTAL_RATE", total_rate),
        ("RB_RATE", rb_rate),
        ("GAME", game),
        ("RATE_MEDAL", medal_rate),
        ("MEDALS", medals),
        ("3ROLLING", rolling3),
        ("5ROLLING", rolling5),
        ("7ROLLING", rolling7),
        ("1RANK", medal_rank1),
        ("3RANK", medal_rank3),
        ("5RANK", medal_rank5),
        ("7RANK", medal_rank7),
    ]
    for label, df_table in labeled_tables:
        df_table.columns = pd.MultiIndex.from_product([[label], df_table.columns])

    # 列を交互に整列して統合・NaN除去・日付ソート・一部データ削除
    interleaved_cols = [
        col
        for col_group in zip(*(df.columns for _, df in labeled_tables))
        for col in col_group
    ]
    merged = pd.concat([df for _, df in labeled_tables], axis=1)[interleaved_cols]
    merged = merged.iloc[:, ::-1]
    # display(merged)
    merged = merged[~merged.iloc[:, 7].isna()]

    # エリアごとに空行挿入して整形
    merged_by_area = pd.DataFrame()
    for area in merged.index.get_level_values("area").unique():
        area_merged = merged.xs(area, level="area", drop_level=False)
        if not area_merged.empty:
            empty_index = pd.MultiIndex.from_tuples(
                [("", " ", " ")], names=merged.index.names
            )
            empty_row = pd.DataFrame(
                [[""] * area_merged.shape[1]],
                index=empty_index,
                columns=area_merged.columns,
            )
            merged_by_area = pd.concat([merged_by_area, area_merged, empty_row])
            
    merged_by_area.to_csv("test.csv")

    # インデックス削除
    merged_by_area = merged_by_area.droplevel("area")
    
    if not merged_by_area.empty:
        empty_index = pd.MultiIndex.from_tuples(
            [(" ", " ")], names=merged_by_area.index.names
        )
        empty_row = pd.DataFrame(
            [[""] * merged_by_area.shape[1]],
            index=empty_index,
            columns=merged_by_area.columns,
        )
        merged_by_model = pd.concat(
            [merged_by_model, merged_by_area, empty_row], axis=0
        )

# 累計した最後の7日を削除
merged_by_model = merged_by_model.iloc[:, :-len(labeled_tables)*6]
merged_by_model.replace([np.inf, -np.inf], np.nan, inplace=True)

sheet_name = "HISTORY"

csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
merged_by_model.to_csv(csv_path)
# add_spreadsheet(spreadsheet, sheet_name, merged_by_model, sheet_clear=True)

merged_by_model.head()

🔍 'EXA FIRST' を含むホール名が見つかりました。
データサイズ: 9880 x 11
📅 検索期間: 2025-03-26 ～ 2025-05-30 📅 抽出期間: 2025-03-26 ～ 2025-05-29
含まれる日数 : 65
データ前処理を行います


7RANK      5RANK      3RANK      1RANK   7ROLLING  \
date               2025-05-29 2025-05-29 2025-05-29 2025-05-29 2025-05-29   
model_name unit_no                                                          
ゴーゴージャグラー3 1001             7          7         10         20    -1614.0   
           1002            12         18          5          8     1032.0   
           1003            14         11         18          6     1482.0   
           1004            20         20         19         19     7334.0   
           1005            18         15         16         15     1873.0   

                     5ROLLING   3ROLLING     MEDALS RATE_MEDAL       GAME  \
date               2025-05-29 2025-05-29 2025-05-29 2025-05-29 2025-05-29   
model_name unit_no                                                          
ゴーゴージャグラー3 1001       -1467.0     -582.0     1453.0      1.145     3345.0   
           1002        1220.0    -1333.0     -468.0      0.887     1382.0   
           1003        -453.0     2150.0     -644.0      0.891     1976.0   
           1004        4863.0     2233.0      971.0      1.068     4768.0   
           1005         258.0      250.0      288.0       1.05     1904.0   

                    ...      1RANK   7ROLLING   5ROLLING   3ROLLING  \
date                ... 2025-04-01 2025-04-01 2025-04-01 2025-04-01   
model_name unit_no  ...                                               
ゴーゴージャグラー3 1001     ...          0        NaN        NaN        NaN   
           1002     ...          0        NaN        NaN        NaN   
           1003     ...          0        NaN        NaN        NaN   
           1004     ...          0        NaN        NaN        NaN   
           1005     ...          0        NaN        NaN        NaN   

                       MEDALS RATE_MEDAL       GAME    RB_RATE TOTAL_RATE  \
date               2025-04-01 2025-04-01 2025-04-01 2025-04-01 2025-04-01   
model_name unit_no                                                          
ゴーゴージャグラー3 1001           NaN        NaN        NaN        NaN        NaN   
           1002           NaN        NaN        NaN        NaN        NaN   
           1003           NaN        NaN        NaN        NaN        NaN   
           1004           NaN        NaN        NaN        NaN        NaN   
           1005           NaN        NaN        NaN        NaN        NaN   

                   GRAPE_RATE  
date               2025-04-01  
model_name unit_no             
ゴーゴージャグラー3 1001           NaN  
           1002           NaN  
           1003           NaN  
           1004           NaN  
           1005           NaN  

[5 rows x 767 columns]

In [ ]:
start_date = datetime.date(today.year, today.month-1, 1) - relativedelta(days=6)
df_db = create_df_from_database(hall_name, start_date, today, model_name="ジャグラー")
df, model_list = df_preprocessing(df_db, hall_name)
df["last_digit"].head()

sheet_name = "SETTING_PREDICT"
csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
df = df[(df["last_digit"] == 1) | (df["last_digit"] == 5)]
index = ["model_name", "unit_no"]
columns = ["date"]
df_pred = df.pivot_table(
    index=index,
    columns=columns,
    values=["5more"],
    # aggfunc="sum",
    # margins=True,
    # margins_name="total",
)
df_pred = df_pred.iloc[:, ::-1]
df_pred = df_pred[~df_pred.iloc[:, 1].isna()]
df_pred.to_csv(csv_path)
df_pred


🔍 'EXA FIRST' を含むホール名が見つかりました。
データサイズ: 9880 x 11
📅 検索期間: 2025-03-26 ～ 2025-05-30 📅 抽出期間: 2025-03-26 ～ 2025-05-29
含まれる日数 : 65
データ前処理を行います


5more                                              \
date                  2025-05-26 2025-05-25 2025-05-21 2025-05-16 2025-05-15   
model_name    unit_no                                                          
アイムジャグラーEX-TP 1100          0.06       0.01       0.00        0.0       0.30   
              1101          0.05       0.00       0.00        0.0       0.00   
              1102          0.02       0.28       0.00        0.0       0.08   
              1103          0.08       0.00       0.37        0.0       0.00   
              1104          0.27       0.19       0.00        0.0       0.00   
...                          ...        ...        ...        ...        ...   
ミスタージャグラー     1109          0.00       0.00       0.00        0.0       0.00   
              1110          0.00       0.00       0.00        0.0       0.00   
              1111          0.00       0.00       0.00        0.0       0.00   
              1112          0.00       0.00       0.00        0.0       0.00   
              1113          0.00       0.00       0.00        0.0       0.00   

                                                                              \
date                  2025-05-11 2025-05-06 2025-05-05 2025-05-01 2025-04-26   
model_name    unit_no                                                          
アイムジャグラーEX-TP 1100          0.02       0.36       0.92       0.01       0.06   
              1101          0.94       0.00       0.01       0.00       0.11   
              1102          0.99       0.00       0.00       0.02       0.32   
              1103          0.15       0.00       0.06       0.10       0.11   
              1104          0.75       0.43       0.05       0.85       0.12   
...                          ...        ...        ...        ...        ...   
ミスタージャグラー     1109          0.00       0.00       0.00       0.00       0.00   
              1110          0.00       0.00       0.00       0.00       0.00   
              1111          0.00       0.00       0.00       0.00       0.00   
              1112          0.00       0.00       0.00       0.00       0.00   
              1113          0.00       0.00       0.00       0.00       0.00   

                                                                              \
date                  2025-04-25 2025-04-21 2025-04-16 2025-04-15 2025-04-11   
model_name    unit_no                                                          
アイムジャグラーEX-TP 1100          0.22       0.21       0.13       0.63        NaN   
              1101          0.00       0.00       0.00       0.00        NaN   
              1102          0.00       0.00       0.00       0.65        NaN   
              1103          0.24       0.00       0.00       0.00        NaN   
              1104          0.00       0.00       0.33       0.14        NaN   
...                          ...        ...        ...        ...        ...   
ミスタージャグラー     1109          0.00       0.00        NaN        NaN        NaN   
              1110          0.00       0.00        NaN        NaN        NaN   
              1111          0.00       0.00        NaN        NaN        NaN   
              1112          0.00       0.00        NaN        NaN        NaN   
              1113          0.00       0.00        NaN        NaN        NaN   

                                                                              
date                  2025-04-06 2025-04-05 2025-04-01 2025-03-31 2025-03-26  
model_name    unit_no                                                         
アイムジャグラーEX-TP 1100           NaN        NaN        NaN        NaN        NaN  
              1101           NaN        NaN        NaN        NaN        NaN  
              1102           NaN        NaN        NaN        NaN        NaN  
              1103           NaN        NaN        NaN        NaN        NaN  
              1104           NaN        NaN        NaN        NaN        NaN  
...                          ...

## MODEL_RATE_HISTORY

In [ ]:

last_month = today - relativedelta(months=0)
month_ago = today - relativedelta(months=1)
last_day = calendar.monthrange(last_month.year, last_month.month)[1]
end_date = datetime.date(last_month.year, last_month.month, last_day)
last_day = calendar.monthrange(month_ago.year, month_ago.month)[1]
start_date = datetime.date(month_ago.year, month_ago.month, 1)

df_db = create_df_from_database(hall_name, start_date, end_date, model_name="ジャグラー")
df, model_list = df_preprocessing(df_db, hall_name)
index = ["model_name", "unit_no"]
columns = ["date"]
merged, details = create_pivot_table(df, index, columns, reverse=True, margins=True)
model_rate = details["medal_rate"].copy()
model_rate = model_rate[~model_rate.iloc[:, 2].isna()]
model_rate.replace([np.inf, -np.inf], np.nan, inplace=True)
model_rate.to_csv(csv_path)
sheet_name = "MODEL_RATE_HISTORY"
csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
add_spreadsheet(spreadsheet, sheet_name, model_rate, sheet_clear=True)
model_rate.head()

🔍 'EXA FIRST' を含むホール名が見つかりました。
データサイズ: 8664 x 11
📅 検索期間: 2025-04-01 ～ 2025-05-31 📅 抽出期間: 2025-04-01 ～ 2025-05-27
含まれる日数 : 57
データ前処理を行います
✅ シート「MODEL_RATE_HISTORY3」が既に存在します。
✅ シート 'MODEL_RATE_HISTORY3' に DataFrame を書き込みました！


MEDAL_RATE                                          \
date                       total 2025-05-27 00:00:00 2025-05-26 00:00:00   
model_name    unit_no                                                      
アイムジャグラーEX-TP 1100         0.989               0.976               0.874   
              1101         1.021               1.185               1.012   
              1102         1.009               0.861               1.095   
              1103         1.000               0.865               0.998   
              1104         0.990               0.919               1.078   

                                                               \
date                  2025-05-25 00:00:00 2025-05-24 00:00:00   
model_name    unit_no                                           
アイムジャグラーEX-TP 1100                  0.919               0.904   
              1101                  0.812               0.735   
              1102                  1.000               0.847   
              1103                  0.943               0.841   
              1104                  0.934               1.085   

                                                               \
date                  2025-05-23 00:00:00 2025-05-22 00:00:00   
model_name    unit_no                                           
アイムジャグラーEX-TP 1100                  0.809               0.762   
              1101                  1.171               1.127   
              1102                  1.042               0.786   
              1103                  1.070               0.967   
              1104                  0.937               0.812   

                                                               \
date                  2025-05-21 00:00:00 2025-05-20 00:00:00   
model_name    unit_no                                           
アイムジャグラーEX-TP 1100                  0.751               1.038   
              1101                  0.969               0.850   
              1102                  1.159               0.958   
              1103                  1.062               1.027   
              1104                  1.036               0.627   

                                           ...                      \
date                  2025-05-19 00:00:00  ... 2025-04-10 00:00:00   
model_name    unit_no                      ...                       
アイムジャグラーEX-TP 1100                  0.969  ...                 NaN   
              1101                  0.880  ...                 NaN   
              1102                  1.245  ...                 NaN   
              1103                  0.969  ...                 NaN   
              1104                  0.791  ...                 NaN   

                                                               \
date                  2025-04-09 00:00:00 2025-04-08 00:00:00   
model_name    unit_no                                           
アイムジャグラーEX-TP 1100                    NaN                 NaN   
              1101                    NaN                 NaN   
              1102                    NaN                 NaN   
              1103                    NaN                 NaN   
              1104                    NaN                 NaN   

                                                               \
date                  2025-04-07 00:00:00 2025-04-06 00:00:00   
model_name    unit_no                                           
アイムジャグラーEX-TP 1100                    NaN                 NaN   
              1101                    NaN                 NaN   
              1102                    NaN                 NaN   
              1103                    NaN                 NaN   
              1104                    NaN                 NaN   

                                                               \
date                  2025-04-05 00:00:00 2025-04-04 00:00:00   
model_name    unit_no                                           
アイムジャグラーEX-TP 1100                    NaN           

In [170]:
rb_rate = details["rb_rate"].copy()
rb_rate = rb_rate[~rb_rate.iloc[:, 2].isna()]
rb_rate.replace([np.inf, -np.inf], np.nan, inplace=True)
rb_rate.to_csv(csv_path)
sheet_name = "RB_RATE_HISTORY"
csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
add_spreadsheet(spreadsheet, sheet_name, rb_rate, sheet_clear=True)
rb_rate.head()

🆕 シート「RB_RATE_HISTORY」を新規作成しました。
✅ シート 'RB_RATE_HISTORY' に DataFrame を書き込みました！


RB_RATE                                          \
date                    total 2025-05-27 00:00:00 2025-05-26 00:00:00   
model_name    unit_no                                                   
アイムジャグラーEX-TP 1100      363.4               374.9               299.3   
              1101      389.5               248.1               403.8   
              1102      395.7               468.3               609.4   
              1103      363.6               589.5               353.4   
              1104      360.6                 NaN               327.0   

                                                               \
date                  2025-05-25 00:00:00 2025-05-24 00:00:00   
model_name    unit_no                                           
アイムジャグラーEX-TP 1100                  451.4               928.0   
              1101                  569.6                 NaN   
              1102                  295.0               494.6   
              1103                  356.4               312.0   
              1104                  279.7               458.3   

                                                               \
date                  2025-05-23 00:00:00 2025-05-22 00:00:00   
model_name    unit_no                                           
アイムジャグラーEX-TP 1100                  301.8               452.7   
              1101                  264.1               326.8   
              1102                  223.6               463.0   
              1103                  212.6               584.0   
              1104                  119.1                 NaN   

                                                               \
date                  2025-05-21 00:00:00 2025-05-20 00:00:00   
model_name    unit_no                                           
アイムジャグラーEX-TP 1100                 1020.0               242.3   
              1101                  636.7               324.0   
              1102                  231.6               559.6   
              1103                  302.9               568.6   
              1104                  418.7               355.0   

                                           ...                      \
date                  2025-05-19 00:00:00  ... 2025-04-10 00:00:00   
model_name    unit_no                      ...                       
アイムジャグラーEX-TP 1100                  324.0  ...                 NaN   
              1101                  921.0  ...                 NaN   
              1102                  298.5  ...                 NaN   
              1103                  271.1  ...                 NaN   
              1104                    NaN  ...                 NaN   

                                                               \
date                  2025-04-09 00:00:00 2025-04-08 00:00:00   
model_name    unit_no                                           
アイムジャグラーEX-TP 1100                    NaN                 NaN   
              1101                    NaN                 NaN   
              1102                    NaN                 NaN   
              1103                    NaN                 NaN   
              1104                    NaN                 NaN   

                                                               \
date                  2025-04-07 00:00:00 2025-04-06 00:00:00   
model_name    unit_no                                           
アイムジャグラーEX-TP 1100                    NaN                 NaN   
              1101                    NaN                 NaN   
              1102                    NaN                 NaN   
              1103                    NaN                 NaN   
              1104                    NaN                 NaN   

                                                               \
date                  2025-04-05 00:00:00 2025-04-04 00:00:00   
model_name    unit_no                                           
アイムジャグラーEX-TP 1100                    NaN                 NaN   
           